In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf
from scipy.spatial import distance as dist

In [2]:
cwd = os.getcwd()
img_path = cwd + "\\sample_input_image.jpg"
vid_path = cwd + "\\pedestrian_survaillance.mp4"
img = cv2.imread(img_path)
classes = ["None", "Person"]

In [3]:
model = tf.saved_model.load("ssd_mobilenet_v1_coco_2018_01_28")
model = model.signatures['serving_default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [4]:
def draw_bounding_boxes(model, image, clas):
    cent = []
    imagenp = image
    width = imagenp.shape[1]
    height = imagenp.shape[0]
    imagenp = np.expand_dims(imagenp, axis=0)
    imagenp = tf.convert_to_tensor(imagenp)
    pred = model(imagenp)

    for x in range(pred['detection_boxes'][0].shape[0]):
        det_score = pred['detection_scores'][0][x].numpy()
        
        if det_score <= 0.0:
            g = image
            continue
        elif det_score >= 0.85:

            ymin,xmin,ymax,xmax=pred['detection_boxes'][0][x].numpy()

            (left, right, top, bottom) = (int(round((xmin * width))), int(round((xmax * width))),
                                          int(round((ymin * height))), int(round((ymax * height))))

            #crop = image[top:bottom, left:right]

            cl = int(pred['detection_classes'][0][x].numpy())


            cx = int(round((right - left)/2)) + left
            cy = int(round((bottom - top)/2)) + top

            cv2.putText(image, "{}, {}%".format(clas[cl], int(round(float(det_score), 2)*100)), 
                        (cx - 50 , cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.circle(image, (cx, cy), 5, (0, 255, 0), -1)
            g = cv2.rectangle(image, (left, top), (right, bottom), color=(0, 255, 0), thickness=1)
            
            cent.append((cx, cy))
                        
    return g, cent



In [ ]:
cv = cv2.VideoCapture(vid_path)
framecount = 0
while cv.isOpened():
    ret, frame = cv.read()
    framecount += 1

    if ret:
        f = draw_bounding_boxes(model, frame, classes)
        cv2.imshow("Reading", f)
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        break
        
cv.release()
cv2. destroyAllWindows() 

In [6]:
f, cent2 = draw_bounding_boxes(model, img, classes)

# try:
#     cv2.imshow("x", f)
#     key = cv2.waitKey(33)
# except:
#     cv2. destroyAllWindows() 

In [10]:
cent = np.array(cent)
cent2 = np.array(cent2)
print(cent)
print(cent2)

[[ 554  372]
 [ 778  347]
 [ 153  370]
 [1037  345]]
[[ 774  343]
 [ 558  367]
 [1036  347]
 [ 149  373]]


In [8]:
Y = dist.cdist(cent, cent2)

In [9]:
print(Y)

[[221.90313202   6.40312424 482.64790479 405.00123457]
 [  5.65685425 220.90722034 258.         629.53713155]
 [621.58667939 405.01111096 883.29949621   5.        ]
 [263.00760445 479.50495305   2.23606798 888.44133177]]


In [15]:
rows = Y.min(axis=1).argsort()
cols = Y.argmin(axis=1)[rows]
print(rows)
print(cols)

[3 2 1 0]
[2 3 0 1]


In [17]:
for aw, awaw in zip(rows, cols):
    print(aw, awaw)

3 2
2 3
1 0
0 1
